In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix
from scipy.spatial import distance
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
DR = 'E'
data_root = DR + r':\addition_yolobbox_json_6'
train_json_folder = f'{data_root}\\train'
val_json_folder = f'{data_root}\\val'  # 검증 데이터셋 폴더 경로
pt_path = DR + r':\project\CVProject\results\result_pt_KDH'

cuda


### mediapipe 랜드마크만 학습
* 비교를 위해 정규화하지 않은 pt파일도 생성 

In [9]:
# 랜드마크 인덱스 정의
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 데이터 증강 함수 정의

# 데이터셋 클래스 정의
class FallSequenceDataset(Dataset):
    def __init__(self, json_files, fit, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        print(f"LANDMARKS length: {len(LANDMARKS)}")
    
        all_landmarks = []
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i:i+self.sequence_length]
                landmarks = []
                fall_frames = 0
                
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y']
                        ])
                    landmarks.append(frame_landmarks)
                    if frame['class'] == 'Fall':
                        fall_frames += 1
                
                all_landmarks.extend(landmarks)

                # 레이블 재정의
                if fall_frames == 0:
                    label = 0  # 비낙상
                elif fall_frames == self.sequence_length:
                    label = 2  # 완전 낙상
                else:
                    label = 1  # 낙상 위험
                
                self.sequences.append(landmarks)
                self.labels.append(label)
        
        if fit == True:
            ## 전체 데이터 정규화
            all_landmarks = np.array(all_landmarks)
            all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
            
            ## 정규화된 데이터를 다시 시퀀스로 재구성
            for i in range(len(self.sequences)):
                start = i * self.sequence_length
                end = start + self.sequence_length
                self.sequences[i] = all_landmarks_scaled[start:end]
        
        if self.sequences:
            print(f"Sample sequence shape: {np.array(self.sequences[0]).shape}")
        else:
            print("No sequences were created.")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if idx >= len(self.sequences):
            raise IndexError(f"Index {idx} out of range. Dataset length: {len(self.sequences)}")
        sequence = self.sequences[idx]
        return torch.FloatTensor(sequence), torch.LongTensor([self.labels[idx]]).squeeze()

# GRU 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout = 0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout = dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[ : , -1, :])
        out = self.fc(out)
        return out

# 데이터 로드 및 전처리
fit_input = input('use fit_transform?: Y/other: ')
if fit_input.lower() == 'y':
    fit = True
else:
    fit = False
train_json_files = [os.path.join(train_json_folder, f) for f in os.listdir(train_json_folder) if f.endswith('.json')]
val_json_files = [os.path.join(val_json_folder, f) for f in os.listdir(val_json_folder) if f.endswith('.json')]

train_dataset = FallSequenceDataset(train_json_files, fit)
val_dataset = FallSequenceDataset(val_json_files, fit)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 클래스 가중치 계산 (train_dataset의 레이블만 사용)
train_labels = [label for _, label in train_dataset]
train_labels = np.array(train_labels, dtype=int)
unique_classes = np.array([0, 1, 2], dtype=int)  # 모든 가능한 클래스를 명시적으로 지정

print("train_labels dtype:", train_labels.dtype)
print("unique_classes dtype:", unique_classes.dtype)
print("Unique labels in train_labels:", np.unique(train_labels))
print("unique_classes:", unique_classes)

class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

if len(train_dataset) > 0:
    sample_sequence, sample_label = train_dataset[0]
    input_size = sample_sequence.shape[1]
    print(f'Input size: {input_size}')
    model = FallDetectionGRU(input_size).to(device)
else:
    print("No data available")
    exit()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001,  weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
num_epochs = 500
best_loss = float('inf')
patience = 20 # 15 이후 다시 실행을 위해 50으로 변경
no_improve = 0
if fit == True:
    file_path = f'{pt_path}\\only_mediapipe.pt'
else:
    file_path = f'{pt_path}\\only_mediapipe_except_normalization.pt'

for epoch in range(num_epochs):
    model.train()
    total_loss_train = 0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss_train = criterion(outputs, labels.view(-1))
        loss_train.backward()
        optimizer.step()
        
        total_loss_train += loss_train.item()
    
    avg_loss_train = total_loss_train / len(train_loader)

    # 검증 단계 추가
    model.eval()
    total_loss_val = 0
    
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val.view(-1))
            total_loss_val += loss_val.item()
    
    avg_loss_val = total_loss_val / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss_train:.4f}, Val Loss: {avg_loss_val:.4f}')
    scheduler.step(avg_loss_val)
    
    if avg_loss_val < best_loss:
        best_loss = avg_loss_val
        no_improve = 0
        torch.save(model.state_dict(), file_path)
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break
    
def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# 학습 루프 내에서 성능 지표 계산
train_f1, train_cm = calculate_metrics(model, train_loader)
val_f1, val_cm = calculate_metrics(model, val_loader)
print(f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
print(f'Train CM:\n{train_cm}\nVal CM:\n{val_cm}')

print("Training completed")

# F1 스코어와 혼동 행렬을 파일로 저장
def save_metrics(f1_train, cm_train, f1_val, cm_val, file_path):
    with open(file_path + '.txt', 'w') as f:
        f.write(f'Train F1: {f1_train:.4f}\n')
        f.write(f'Val F1: {f1_val:.4f}\n')
        f.write(f'Train Confusion Matrix:\n{cm_train}\n')
        f.write(f'Val Confusion Matrix:\n{cm_val}\n')

# 결과 저장
save_metrics(train_f1, train_cm, val_f1, val_cm, file_path)

print("저장 완료")

LANDMARKS length: 11
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C5.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C6.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C7.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C8.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C1.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C2.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C3.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C4.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C5.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C6.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C7.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C8.json
Processing file: E:\addition_yolobbox_json_6\train\00093_H_A_FY_C1.json
Processing file: E:\addition_yolobbox_json_

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/500], Train Loss: 0.7414, Val Loss: 0.5799
Epoch [2/500], Train Loss: 0.7113, Val Loss: 0.5341
Epoch [3/500], Train Loss: 0.6943, Val Loss: 0.5484
Epoch [4/500], Train Loss: 0.6872, Val Loss: 0.5298
Epoch [5/500], Train Loss: 0.6827, Val Loss: 0.5192
Epoch [6/500], Train Loss: 0.6779, Val Loss: 0.5170
Epoch [7/500], Train Loss: 0.6778, Val Loss: 0.5070
Epoch [8/500], Train Loss: 0.6727, Val Loss: 0.5113
Epoch [9/500], Train Loss: 0.6712, Val Loss: 0.5104
Epoch [10/500], Train Loss: 0.6680, Val Loss: 0.5056
Epoch [11/500], Train Loss: 0.6640, Val Loss: 0.4927
Epoch [12/500], Train Loss: 0.6572, Val Loss: 0.4904
Epoch [13/500], Train Loss: 0.6550, Val Loss: 0.4869
Epoch [14/500], Train Loss: 0.6472, Val Loss: 0.4671
Epoch [15/500], Train Loss: 0.6418, Val Loss: 0.4662
Epoch [16/500], Train Loss: 0.6370, Val Loss: 0.4902
Epoch [17/500], Train Loss: 0.6340, Val Loss: 0.4941
Epoch [18/500], Train Loss: 0.6314, Val Loss: 0.4820
Epoch [19/500], Train Loss: 0.6285, Val Loss: 0.4753
Ep

### sensordata를 포함해 클래스를 재정의한 json 파일로 다시 훈련
* 데이터 증강은 input_size가 늘어나 삭제
* sensordata의 fall_start_frame과 fall_end_frame을 'danger' 클래스로
* mediapipe의 랜드마크와 재정의된 클래스 학습
* 비교를 위해 정규화하지 않은 .pt파일도 생성

In [4]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 데이터 증강 삭제
# 데이터셋 클래스 정의
class FallSequenceDataset(Dataset):
    def __init__(self, json_files, fit, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        print(f"LANDMARKS length: {len(LANDMARKS)}")
        
        if self.sequences:
            print(f"sequence shape : {self.sequences[0].shape}")
            print(f"랜드마크 수 : {len(LANDMARKS)}")
            print(f"Features / landmark: {self.sequences[0].shape[1] // len(LANDMARKS)}")
            print(f"전체 features per frame: {self.sequences[0].shape[1]}")
        else:
            print("시퀀스 생성 실패")
    
        all_landmarks = []
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i:i+self.sequence_length]
                landmarks = []
                
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y']
                        ])
                    landmarks.append(frame_landmarks)
                
                all_landmarks.extend(landmarks)
                
                # 마지막 프레임의 클래스를 레이블로 사용
                label = 0 if frame['class'] == 'Normal' else (1 if frame['class'] == 'Danger' else 2)
                
                self.sequences.append(landmarks)
                self.labels.append(label)
        
        if fit == True:
            ## 전체 데이터 정규화
            all_landmarks = np.array(all_landmarks)
            all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
            
            ## 정규화된 데이터를 다시 시퀀스로 재구성
            for i in range(len(self.sequences)):
                start = i * self.sequence_length
                end = start + self.sequence_length
                self.sequences[i] = all_landmarks_scaled[start:end]
        
        if self.sequences:
            print(f"sequence shape: {len(self.sequences[0])}")
        else:
            print("sequences 생성 실패")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if idx >= len(self.sequences):
            raise IndexError(f"Index {idx} out of range. Dataset length: {len(self.sequences)}")
        sequence = self.sequences[idx]
        return torch.FloatTensor(sequence), torch.LongTensor([self.labels[idx]]).squeeze()

# GRU 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout=0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# 데이터 로드 및 전처리
fit_input = input('use fit_transform?: Y/other: ')
if fit_input.lower() == 'y':
    fit = True
else:
    fit = False

train_json_files = [os.path.join(train_json_folder, f) for f in os.listdir(train_json_folder) if f.endswith('.json')]
val_json_files = [os.path.join(val_json_folder, f) for f in os.listdir(val_json_folder) if f.endswith('.json')]

train_dataset = FallSequenceDataset(train_json_files, fit)
val_dataset = FallSequenceDataset(val_json_files, fit)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 클래스 가중치 계산 (train_dataset의 레이블만 사용)
train_labels = [label for _, label in train_dataset]
train_labels = np.array(train_labels, dtype=int)
unique_classes = np.array([0, 1, 2], dtype=int)  # 모든 가능한 클래스를 명시적으로 지정

print("train_labels dtype:", train_labels.dtype)
print("unique_classes dtype:", unique_classes.dtype)
print("Unique labels in train_labels:", np.unique(train_labels))
print("unique_classes:", unique_classes)

class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

if len(train_dataset) > 0:
    sample_sequence, sample_label = train_dataset[0]
    input_size = sample_sequence.shape[1]
    print(f'Input size: {input_size}')
    model = FallDetectionGRU(input_size).to(device)
else:
    print("No data available")
    exit()
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
num_epochs = 500
best_loss = float('inf')
patience = 15
no_improve = 0

if fit == True:
    file_path = f'{pt_path}\\mediapipe_sensordata.pt'
else:
    file_path = f'{pt_path}\\mediapipe_sensordata_except_normalization.pt'

for epoch in range(num_epochs):
    model.train()
    total_loss_train = 0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss_train = criterion(outputs, labels.view(-1))
        loss_train.backward()
        optimizer.step()
        
        total_loss_train += loss_train.item()
    
    avg_loss_train = total_loss_train / len(train_loader)

    # 검증 단계
    model.eval()
    total_loss_val = 0
    
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val.view(-1))
            total_loss_val += loss_val.item()
    
    avg_loss_val = total_loss_val / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss_train:.4f}, Val Loss: {avg_loss_val:.4f}')
    scheduler.step(avg_loss_val)
    
    if avg_loss_val < best_loss:
        best_loss = avg_loss_val
        no_improve = 0
        torch.save(model.state_dict(), file_path)
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break

def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# 학습 루프 내에서 성능 지표 계산
train_f1, train_cm = calculate_metrics(model, train_loader)
val_f1, val_cm = calculate_metrics(model, val_loader)
print(f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
print(f'Train CM:\n{train_cm}\nVal CM:\n{val_cm}')

print("학습 완료")

# F1 스코어와 혼동 행렬을 파일로 저장
def save_metrics(f1_train, cm_train, f1_val, cm_val, file_path):
    with open(file_path + '.txt', 'w') as f:
        f.write(f'Train F1: {f1_train:.4f}\n')
        f.write(f'Val F1: {f1_val:.4f}\n')
        f.write(f'Train Confusion Matrix:\n{cm_train}\n')
        f.write(f'Val Confusion Matrix:\n{cm_val}\n')

# 결과 저장
save_metrics(train_f1, train_cm, val_f1, val_cm, file_path)

print("저장 완료")

LANDMARKS length: 11
시퀀스 생성 실패
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C5.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C6.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C7.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C8.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C1.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C2.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C3.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C4.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C5.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C6.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C7.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C8.json
Processing file: E:\addition_yolobbox_json_6\train\00093_H_A_FY_C1.json
Processing file: E:\addition_yolo

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/500], Train Loss: 0.7442, Val Loss: 0.4909
Epoch [2/500], Train Loss: 0.6203, Val Loss: 0.4873
Epoch [3/500], Train Loss: 0.5878, Val Loss: 0.4731
Epoch [4/500], Train Loss: 0.5689, Val Loss: 0.4080
Epoch [5/500], Train Loss: 0.5559, Val Loss: 0.4056
Epoch [6/500], Train Loss: 0.5490, Val Loss: 0.3934
Epoch [7/500], Train Loss: 0.5415, Val Loss: 0.4199
Epoch [8/500], Train Loss: 0.5351, Val Loss: 0.4256
Epoch [9/500], Train Loss: 0.5277, Val Loss: 0.4241
Epoch [10/500], Train Loss: 0.5223, Val Loss: 0.3990
Epoch [11/500], Train Loss: 0.5180, Val Loss: 0.4256
Epoch [12/500], Train Loss: 0.5159, Val Loss: 0.3850
Epoch [13/500], Train Loss: 0.5132, Val Loss: 0.4408
Epoch [14/500], Train Loss: 0.5112, Val Loss: 0.3747
Epoch [15/500], Train Loss: 0.5055, Val Loss: 0.3943
Epoch [16/500], Train Loss: 0.5032, Val Loss: 0.3746
Epoch [17/500], Train Loss: 0.5006, Val Loss: 0.3755
Epoch [18/500], Train Loss: 0.4992, Val Loss: 0.3791
Epoch [19/500], Train Loss: 0.4969, Val Loss: 0.3890
Ep

### sensordata와 yolo bbox 좌표를 동시에 training
* 클래스 예측은 랜드마크의 좌표값을 우선적으로, 차선으로 bbox 좌표로 분류하도록 수정
* mediapipe의 랜드마크, YOLO bbox의 좌표, bbox의 비율 학습
* 비교를 위해 정규화하지 않은 .pt파일도 생성

In [8]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# bbox의 비율 계산 함수
def bbox_ratio(bbox) : 
    w = bbox['x2'] - bbox['x1']
    h = bbox['y2'] - bbox['y1']
    return w / h if h != 0 else 0
    
# 데이터셋 클래스 정의
class FallSequenceDataset(Dataset):
    def __init__(self, json_files, fit, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        print(f"LANDMARKS length: {len(LANDMARKS)}")
           
        all_landmarks = []
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i:i+self.sequence_length]
                landmarks = []
                
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y']
                        ])
                        
                    # YOLO bbox 좌표 추가
                    try:
                        bbox = frame['bbox']
                        frame_landmarks.extend([
                            frame['bbox']['x1'],
                            frame['bbox']['y1'],
                            frame['bbox']['x2'],
                            frame['bbox']['y2']
                        ])
                        # bbox 비율을 feature로 추가
                        bbox_ratio_value = bbox_ratio(bbox)
                        frame_landmarks.append(bbox_ratio_value)                       
                    except KeyError:
                        print('bbox 정보 없음. 기본값으로 사용')
                        frame_landmarks.extend([0, 0, 1, 1, 0])  # 기본값 사용
                        
                    landmarks.append(frame_landmarks)
                
                all_landmarks.extend(landmarks)
                
                # 마지막 프레임의 클래스를 레이블로 사용
                label = 0 if frame['class'] == 'Normal' else (1 if frame['class'] == 'Danger' else 2)
                
                self.sequences.append(landmarks)
                self.labels.append(label)
        
        if fit == True:
            ## 전체 데이터 정규화
            all_landmarks = np.array(all_landmarks)
            all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
            
            ## 정규화된 데이터를 다시 시퀀스로 재구성
            for i in range(len(self.sequences)):
                start = i * self.sequence_length
                end = start + self.sequence_length
                self.sequences[i] = all_landmarks_scaled[start:end]
        
        if self.sequences:
            print(f"sequence shape: {len(self.sequences[0])}")
            print(f"Features per frame: {len(self.sequences[0][1])}")
        else:
            print("sequences 생성 실패")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if idx >= len(self.sequences):
            raise IndexError(f"Index {idx} out of range. Dataset length: {len(self.sequences)}")
        sequence = self.sequences[idx]
        return torch.FloatTensor(sequence), torch.LongTensor([self.labels[idx]]).squeeze()

# GRU 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers = 2, num_classes = 3, output_size=3, dropout = 0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# 데이터 로드 및 전처리
fit_input = input('use fit_transform?: Y/other: ')
if fit_input.lower() == 'y':
    fit = True
else:
    fit = False
    
train_json_files = [os.path.join(train_json_folder, f) for f in os.listdir(train_json_folder) if f.endswith('.json')]
val_json_files = [os.path.join(val_json_folder, f) for f in os.listdir(val_json_folder) if f.endswith('.json')]

train_dataset = FallSequenceDataset(train_json_files, fit)
val_dataset = FallSequenceDataset(val_json_files, fit)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 클래스 가중치 계산 (train_dataset의 레이블만 사용)
train_labels = [label for _, label in train_dataset]
train_labels = np.array(train_labels, dtype=int)
unique_classes = np.array([0, 1, 2], dtype=int)  # 모든 가능한 클래스를 명시적으로 지정

print("train_labels dtype:", train_labels.dtype)
print("unique_classes dtype:", unique_classes.dtype)
print("Unique labels in train_labels:", np.unique(train_labels))
print("unique_classes:", unique_classes)

class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

if len(train_dataset) > 0:
    sample_sequence, sample_label = train_dataset[0]
    input_size = sample_sequence.shape[1]
    print(f'Input size: {input_size}')
    model = FallDetectionGRU(input_size).to(device)
else:
    print("No data available")
    exit()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
num_epochs = 500
best_loss = float('inf')
patience = 15
no_improve = 0

if fit == True:
    file_path = f'{pt_path}\\mediapipe_sensordata_bbox.pt'
else:
    file_path = f'{pt_path}\\mediapipe_sensordata_bbox_except_normalization.pt'

for epoch in range(num_epochs):
    model.train()
    total_loss_train = 0
    
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss_train = criterion(outputs, labels.view(-1))
        loss_train.backward()
        optimizer.step()
        
        total_loss_train += loss_train.item()
    
    avg_loss_train = total_loss_train / len(train_loader)

    # 검증 단계
    model.eval()
    total_loss_val = 0
    
    with torch.no_grad():
        for sequences_val, labels_val in val_loader:
            sequences_val, labels_val = sequences_val.to(device), labels_val.to(device)
            outputs_val = model(sequences_val)
            loss_val = criterion(outputs_val, labels_val.view(-1))
            total_loss_val += loss_val.item()
    
    avg_loss_val = total_loss_val / len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_loss_train:.4f}, Val Loss: {avg_loss_val:.4f}')
    scheduler.step(avg_loss_val)
    
    if avg_loss_val < best_loss:
        best_loss = avg_loss_val
        no_improve = 0
        torch.save(model.state_dict(), file_path)
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break

def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# 학습 루프 내에서 성능 지표 계산
train_f1, train_cm = calculate_metrics(model, train_loader)
val_f1, val_cm = calculate_metrics(model, val_loader)
print(f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
print(f'Train CM:\n{train_cm}\nVal CM:\n{val_cm}')

print("학습 완료")

# F1 스코어와 혼동 행렬을 파일로 저장
def save_metrics(f1_train, cm_train, f1_val, cm_val, file_path):
    with open(file_path + '.txt', 'w') as f:
        f.write(f'Train F1: {f1_train:.4f}\n')
        f.write(f'Val F1: {f1_val:.4f}\n')
        f.write(f'Train Confusion Matrix:\n{cm_train}\n')
        f.write(f'Val Confusion Matrix:\n{cm_val}\n')

# 결과 저장
save_metrics(train_f1, train_cm, val_f1, val_cm, file_path)

print("저장 완료")

LANDMARKS length: 11
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C5.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C6.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C7.json
Processing file: E:\addition_yolobbox_json_6\train\02970_L_F_FY_C8.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C1.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C2.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C3.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C4.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C5.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C6.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C7.json
Processing file: E:\addition_yolobbox_json_6\train\00060_H_A_SY_C8.json
Processing file: E:\addition_yolobbox_json_6\train\00093_H_A_FY_C1.json
Processing file: E:\addition_yolobbox_json_

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/500], Train Loss: 0.8279, Val Loss: 0.6686
Epoch [2/500], Train Loss: 0.8051, Val Loss: 0.6413
Epoch [3/500], Train Loss: 0.8013, Val Loss: 0.6492
Epoch [4/500], Train Loss: 0.7977, Val Loss: 0.6582
Epoch [5/500], Train Loss: 0.8035, Val Loss: 0.6459
Epoch [6/500], Train Loss: 0.7968, Val Loss: 0.6690
Epoch [7/500], Train Loss: 0.7968, Val Loss: 0.6777
Epoch [8/500], Train Loss: 0.8032, Val Loss: 0.6703
Epoch [9/500], Train Loss: 0.7742, Val Loss: 0.6633
Epoch [10/500], Train Loss: 0.7705, Val Loss: 0.6411
Epoch [11/500], Train Loss: 0.7676, Val Loss: 0.6488
Epoch [12/500], Train Loss: 0.7644, Val Loss: 0.6557
Epoch [13/500], Train Loss: 0.7581, Val Loss: 0.6530
Epoch [14/500], Train Loss: 0.7481, Val Loss: 0.6065
Epoch [15/500], Train Loss: 0.7354, Val Loss: 0.6125
Epoch [16/500], Train Loss: 0.7280, Val Loss: 0.6032
Epoch [17/500], Train Loss: 0.7249, Val Loss: 0.5957
Epoch [18/500], Train Loss: 0.7166, Val Loss: 0.5982
Epoch [19/500], Train Loss: 0.7137, Val Loss: 0.6882
Ep

### mediapipe 랜드마크, bbox의 ratio, bbox의 class, 속도 학습
* 시퀀스를 기반으로 클래스 우선 분류, bbox의 ratio로 최종 클래스 결정
* mediapipe의 랜드마크, bbox의 비율, 상체의 속도(머리, 양쪽 어깨) 학습
* 비교를 위해 정규화하지 않은 .pt파일도 생성

In [ ]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

def calculate_head_upper_body_speed(current_frame, prev_frame):
    h = np.array([current_frame['landmark_0']['x'], current_frame['landmark_0']['y']])
    l = np.array([current_frame['landmark_11']['x'], current_frame['landmark_11']['y']])
    r = np.array([current_frame['landmark_12']['x'], current_frame['landmark_12']['y']])
    
    prev_h = np.array([prev_frame['landmark_0']['x'], prev_frame['landmark_0']['y']])
    prev_l = np.array([prev_frame['landmark_11']['x'], prev_frame['landmark_11']['y']])
    prev_r = np.array([prev_frame['landmark_12']['x'], prev_frame['landmark_12']['y']])
    
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3
    
    return distance.euclidean(center_new, center_prev)

class FallSequenceDataset(Dataset):
    def __init__(self, json_files, fit, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        self.class_mapping = {0: 'Normal', 1: 'Danger', 2: 'Fall'}
        
        all_landmarks = []
        for json_file in tqdm(json_files, desc="Processing JSON files"):
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                frames = list(data['pose_data'].values())
                fall_start = data.get('fall_start_frame', None)
                fall_end = data.get('fall_end_frame', float('inf'))
                
                for i in range(0, len(frames) - self.sequence_length + 1):
                    sequence = frames[i:i+self.sequence_length]
                    landmarks = []
                    for j, frame in enumerate(sequence):
                        frame_landmarks = []
                        for landmark in LANDMARKS:
                            if f'landmark_{landmark}' not in frame:
                                print(f"Missing landmark {landmark} in frame")
                                continue
                            frame_landmarks.extend([
                                frame[f'landmark_{landmark}']['x'],
                                frame[f'landmark_{landmark}']['y']
                            ])
                        
                        bbox = frame.get('bbox')
                        if bbox:
                            # bbox_ratio 추가
                            bbox_width = bbox['x2'] - bbox['x1']
                            bbox_height = bbox['y2'] - bbox['y1']
                            bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else 1.0
                            # bbox_class 설정
                            if bbox_ratio <= 0.7:
                                b_label = 0  # Normal
                            elif 0.7 < bbox_ratio <= 0.8:
                                b_label = 1  # Danger
                            else:
                                b_label = 2  # Fall                           
                            frame_landmarks.extend([bbox_ratio, b_label])
                        
                        else:
                            frame_landmarks.extend([1.0, 0])
                        
                        if j > 0:
                            head_torso_speed = calculate_head_upper_body_speed(sequence[j], sequence[j-1])
                        else:
                            head_torso_speed = 0.0
                        frame_landmarks.append(head_torso_speed)
                        
                        landmarks.append(frame_landmarks)
                    
                    last_frame_index = i + self.sequence_length - 1
                    if fall_start is not None and fall_end is not None:
                        if fall_start <= last_frame_index < fall_end:
                            label = 1  # Danger
                        elif last_frame_index >= fall_end:
                            label = 2  # Fall
                        else:
                            label = 0  # Normal
                    else:
                        label = 0 if frame['class'] == 'Normal' else (2 if frame['class'] == 'Fall' else 1)
                    self.labels.append(label)
                    
                    
                        
                    self.sequences.append(landmarks)
                    all_landmarks.extend(landmarks)
            except Exception as e:
                print(f"Error processing file {json_file}: {e}")
                continue
        
        if not self.sequences:
            raise ValueError("No valid sequences found in the dataset")
        
        if fit == True:
            ## 전체 데이터 정규화
            all_landmarks = np.array(all_landmarks)
            all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
            
            ## 정규화된 데이터를 다시 시퀀스로 재구성
            for i in range(len(self.sequences)):
                start = i * self.sequence_length
                end = start + self.sequence_length
                self.sequences[i] = all_landmarks_scaled[start:end]
        
        print(f"Total sequences: {len(self.sequences)}")
        print(f"Labels distribution: {np.bincount(self.labels)}")
        print(f"Sequence shape: {len(self.sequences[0])}")
        print(f"Features per frame: {len(self.sequences[0][1])}")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.sequences[idx]), torch.LongTensor([self.labels[idx]]).squeeze()

class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=3, dropout=0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

def train_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for sequences, labels in tqdm(data_loader, desc="Training"):
        sequences, labels = sequences.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def validate_epoch(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for sequences, labels in tqdm(data_loader, desc="Validating"):
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels.view(-1))
            total_loss += loss.item()
    return total_loss / len(data_loader)

def calculate_metrics(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# F1 스코어와 혼동 행렬을 파일로 저장
def save_metrics(f1_train, cm_train, f1_val, cm_val, file_path):
    with open(file_path + '.txt', 'w') as f:
        f.write(f'Train F1: {f1_train:.4f}\n')
        f.write(f'Val F1: {f1_val:.4f}\n')
        f.write(f'Train Confusion Matrix:\n{cm_train}\n')
        f.write(f'Val Confusion Matrix:\n{cm_val}\n')


fit_input = input('use fit_transform?: Y/other: ')
if fit_input.lower() == 'y':
    fit = True
else:
    fit = False
train_json_files = [os.path.join(train_json_folder, f) for f in os.listdir(train_json_folder) if f.endswith('.json')]
val_json_files = [os.path.join(val_json_folder, f) for f in os.listdir(val_json_folder) if f.endswith('.json')]

train_dataset = FallSequenceDataset(train_json_files, fit)
val_dataset = FallSequenceDataset(val_json_files, fit)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 클래스 가중치 계산 (train_dataset의 레이블만 사용)
train_labels = [label for _, label in train_dataset]
train_labels = np.array(train_labels, dtype=int)
unique_classes = np.array([0, 1, 2], dtype=int)  # 모든 가능한 클래스를 명시적으로 지정

print("train_labels dtype:", train_labels.dtype)
print("unique_classes dtype:", unique_classes.dtype)
print("Unique labels in train_labels:", np.unique(train_labels))
print("unique_classes:", unique_classes)

class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

if len(train_dataset) > 0:
    sample_sequence, sample_label = train_dataset[0]
    input_size = sample_sequence.shape[1]
    print(f'Input size: {input_size}')
    model = FallDetectionGRU(input_size).to(device)
else:
    print("No data available")
    exit(0)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

num_epochs = 150
best_loss = float('inf')
patience = 20
no_improve = 0

if fit == True:
    file_path = f'{pt_path}\\mediapipe_sensordata_ratio_speed.pt'
else:
    file_path = f'{pt_path}\\mediapipe_sensordata_bbox_speed_except_normalizaion.pt'

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = validate_epoch(model, val_loader, criterion, device)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    scheduler.step(val_loss)
    
    if val_loss < best_loss:
        best_loss = val_loss
        no_improve = 0
        torch.save(model.state_dict(), file_path)
        print(f"Best model saved: {best_loss:.4f}")
    else:
        no_improve += 1
        if no_improve >= patience:
            print("Early stopping")
            break

model.load_state_dict(torch.load(file_path))
train_f1, train_cm = calculate_metrics(model, train_loader, device)
val_f1, val_cm = calculate_metrics(model, val_loader, device)

print(f'Train F1: {train_f1:.4f}')
print(f'Train CM:\n{train_cm}')
print(f'Validation F1: {val_f1:.4f}')
print(f'Validation CM:\n{val_cm}')

print("Training completed")

# 결과 저장
save_metrics(train_f1, train_cm, val_f1, val_cm, file_path)

print("저장 완료")

Processing JSON files: 100%|██████████| 1800/1800 [00:07<00:00, 252.45it/s]


Total sequences: 174083
Labels distribution: [100107  14080  59896]
Sequence shape: 3
Features per frame: 25


Processing JSON files: 100%|██████████| 224/224 [00:00<00:00, 278.78it/s]


Total sequences: 21587
Labels distribution: [12064  1770  7753]
Sequence shape: 3
Features per frame: 25


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Epoch [43/150], Train Loss: 0.3640, Val Loss: 0.2878


Validating: 100%|██████████| 675/675 [00:00<00:00, 1214.03it/s]


Epoch [44/150], Train Loss: 0.3635, Val Loss: 0.2878


Validating: 100%|██████████| 675/675 [00:00<00:00, 1171.88it/s]
C:\Users\user\AppData\Local\Temp\ipykernel_13616\941878484.py:254: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature

Epoch [45/150], Train Loss: 0.3643, Val Loss: 0.2878
Early stopping
Train F1: 0.8817
Train CM:
[[85628  8095  6384]
 [ 1615 11337  1128]
 [ 1834  2657 55405]]
Validation F1: 0.9024
Validation CM:
[[10852   797   415]
 [  193  1432   145]
 [  319   346  7088]]
Training completed
저장 완료


### mediapipe 랜드마크, bbox의 xy좌표, bbox의 ratio, bbox의 class, 속도 학습
* 시퀀스를 기반으로 클래스 우선 분류, bbox의 ratio로 최종 클래스 결정
* mediapipe의 랜드마크, bbox의 xy좌표, bbox의 비율, 상체의 속도(머리, 양쪽 어깨) 학습
* 비교를 위해 정규화하지 않은 .pt파일도 생성

In [8]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

def calculate_head_upper_body_speed(current_frame, prev_frame):
    h = np.array([current_frame['landmark_0']['x'], current_frame['landmark_0']['y']])
    l = np.array([current_frame['landmark_11']['x'], current_frame['landmark_11']['y']])
    r = np.array([current_frame['landmark_12']['x'], current_frame['landmark_12']['y']])
    
    prev_h = np.array([prev_frame['landmark_0']['x'], prev_frame['landmark_0']['y']])
    prev_l = np.array([prev_frame['landmark_11']['x'], prev_frame['landmark_11']['y']])
    prev_r = np.array([prev_frame['landmark_12']['x'], prev_frame['landmark_12']['y']])
    
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3
    
    return distance.euclidean(center_new, center_prev)

class FallSequenceDataset(Dataset):
    def __init__(self, json_files, fit, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        self.class_mapping = {0: 'Normal', 1: 'Danger', 2: 'Fall'}
        
        all_landmarks = []
        for json_file in tqdm(json_files, desc="Processing JSON files"):
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                frames = list(data['pose_data'].values())
                fall_start = data.get('fall_start_frame', None)
                fall_end = data.get('fall_end_frame', float('inf'))
                
                for i in range(0, len(frames) - self.sequence_length + 1):
                    sequence = frames[i:i+self.sequence_length]
                    landmarks = []
                    for j, frame in enumerate(sequence):
                        frame_landmarks = []
                        for landmark in LANDMARKS:
                            if f'landmark_{landmark}' not in frame:
                                print(f"Missing landmark {landmark} in frame")
                                continue
                            frame_landmarks.extend([
                                frame[f'landmark_{landmark}']['x'],
                                frame[f'landmark_{landmark}']['y']
                            ])
                        
                        bbox = frame.get('bbox')
                        if bbox:
                            # bbox_ratio 추가
                            bbox_width = bbox['x2'] - bbox['x1']
                            bbox_height = bbox['y2'] - bbox['y1']
                            bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else 1.0
                            # bbox_class 설정
                            if bbox_ratio <= 0.7:
                                b_label = 0  # Normal
                            elif 0.7 < bbox_ratio <= 0.8:
                                b_label = 1  # Danger
                            else:
                                b_label = 2  # Fall
                            frame_landmarks.extend([bbox['x1'], bbox['y1'], bbox['x2'], bbox['y2'], bbox_ratio, b_label])
                        
                        else:
                            frame_landmarks.extend([0, 0, 1, 1, 1.0, 0])
                        
                        if j > 0:
                            head_torso_speed = calculate_head_upper_body_speed(sequence[j], sequence[j-1])
                        else:
                            head_torso_speed = 0.0
                        frame_landmarks.append(head_torso_speed)
                        
                        landmarks.append(frame_landmarks)
                    
                    last_frame_index = i + self.sequence_length - 1
                    if fall_start is not None and fall_end is not None:
                        if fall_start <= last_frame_index < fall_end:
                            label = 1  # Danger
                        elif last_frame_index >= fall_end:
                            label = 2  # Fall
                        else:
                            label = 0  # Normal
                    else:
                        label = 0 if frame['class'] == 'Normal' else (2 if frame['class'] == 'Fall' else 1)
                    self.labels.append(label)
                    
                    
                        
                    self.sequences.append(landmarks)
                    all_landmarks.extend(landmarks)
            except Exception as e:
                print(f"Error processing file {json_file}: {e}")
                continue
        
        if not self.sequences:
            raise ValueError("No valid sequences found in the dataset")
        
        if fit == True:
            ## 전체 데이터 정규화
            all_landmarks = np.array(all_landmarks)
            all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
            
            ## 정규화된 데이터를 다시 시퀀스로 재구성
            for i in range(len(self.sequences)):
                start = i * self.sequence_length
                end = start + self.sequence_length
                self.sequences[i] = all_landmarks_scaled[start:end]
        
        print(f"Total sequences: {len(self.sequences)}")
        print(f"Labels distribution: {np.bincount(self.labels)}")
        print(f"Sequence shape: {len(self.sequences[0])}")
        print(f"Features per frame: {len(self.sequences[0][1])}")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.sequences[idx]), torch.LongTensor([self.labels[idx]]).squeeze()

class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=3, dropout=0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

def train_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for sequences, labels in tqdm(data_loader, desc="Training"):
        sequences, labels = sequences.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def validate_epoch(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for sequences, labels in tqdm(data_loader, desc="Validating"):
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels.view(-1))
            total_loss += loss.item()
    return total_loss / len(data_loader)

def calculate_metrics(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

# F1 스코어와 혼동 행렬을 파일로 저장
def save_metrics(f1_train, cm_train, f1_val, cm_val, file_path):
    with open(file_path + '.txt', 'w') as f:
        f.write(f'Train F1: {f1_train:.4f}\n')
        f.write(f'Val F1: {f1_val:.4f}\n')
        f.write(f'Train Confusion Matrix:\n{cm_train}\n')
        f.write(f'Val Confusion Matrix:\n{cm_val}\n')


fit_input = input('use fit_transform?: Y/other: ')
if fit_input.lower() == 'y':
    fit = True
else:
    fit = False
train_json_files = [os.path.join(train_json_folder, f) for f in os.listdir(train_json_folder) if f.endswith('.json')]
val_json_files = [os.path.join(val_json_folder, f) for f in os.listdir(val_json_folder) if f.endswith('.json')]

train_dataset = FallSequenceDataset(train_json_files, fit)
val_dataset = FallSequenceDataset(val_json_files, fit)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 클래스 가중치 계산 (train_dataset의 레이블만 사용)
train_labels = [label for _, label in train_dataset]
train_labels = np.array(train_labels, dtype=int)
unique_classes = np.array([0, 1, 2], dtype=int)  # 모든 가능한 클래스를 명시적으로 지정

print("train_labels dtype:", train_labels.dtype)
print("unique_classes dtype:", unique_classes.dtype)
print("Unique labels in train_labels:", np.unique(train_labels))
print("unique_classes:", unique_classes)

class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_labels)
class_weights = torch.FloatTensor(class_weights).to(device)

# 손실 함수에 가중치 적용
criterion = nn.CrossEntropyLoss(weight=class_weights)

if len(train_dataset) > 0:
    sample_sequence, sample_label = train_dataset[0]
    input_size = sample_sequence.shape[1]
    print(f'Input size: {input_size}')
    model = FallDetectionGRU(input_size).to(device)
else:
    print("No data available")
    exit(0)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

num_epochs = 150
best_loss = float('inf')
patience = 20
no_improve = 0

if fit == True:
    file_path = f'{pt_path}\\mediapipe_sensordata_bbox_ratio_speed.pt'
else:
    file_path = f'{pt_path}\\mediapipe_sensordata_bbox_ratio_speed_except_normalizaion.pt'

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss = validate_epoch(model, val_loader, criterion, device)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    scheduler.step(val_loss)
    
    if val_loss < best_loss:
        best_loss = val_loss
        no_improve = 0
        torch.save(model.state_dict(), file_path)
        print(f"Best model saved: {best_loss:.4f}")
    else:
        no_improve += 1
        if no_improve >= patience:
            print("Early stopping")
            break

model.load_state_dict(torch.load(file_path))
train_f1, train_cm = calculate_metrics(model, train_loader, device)
val_f1, val_cm = calculate_metrics(model, val_loader, device)

print(f'Train F1: {train_f1:.4f}')
print(f'Train CM:\n{train_cm}')
print(f'Validation F1: {val_f1:.4f}')
print(f'Validation CM:\n{val_cm}')

print("Training completed")

# 결과 저장
save_metrics(train_f1, train_cm, val_f1, val_cm, file_path)

print("저장 완료")

Processing JSON files: 100%|██████████| 1800/1800 [00:06<00:00, 263.83it/s]


Total sequences: 174083
Labels distribution: [100107  14080  59896]
Sequence shape: 3
Features per frame: 29


Processing JSON files: 100%|██████████| 224/224 [00:00<00:00, 235.79it/s]


Total sequences: 21587
Labels distribution: [12064  1770  7753]
Sequence shape: 3
Features per frame: 29


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Epoch [57/150], Train Loss: 0.7316, Val Loss: 0.5995


Validating: 100%|██████████| 675/675 [00:00<00:00, 1188.12it/s]


Epoch [58/150], Train Loss: 0.7309, Val Loss: 0.5993


Validating: 100%|██████████| 675/675 [00:00<00:00, 1053.85it/s]
C:\Users\user\AppData\Local\Temp\ipykernel_13616\3353674359.py:254: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

Epoch [59/150], Train Loss: 0.7297, Val Loss: 0.5991
Early stopping
Train F1: 0.7906
Train CM:
[[78443  7142 14522]
 [ 4170  3386  6524]
 [ 2427  1224 56245]]
Validation F1: 0.8341
Validation CM:
[[10468   708   888]
 [  510   411   849]
 [  323   171  7259]]
Training completed
저장 완료
